In [10]:
#importing the required libraries
import os
import time
from ast import literal_eval
from datetime import datetime
import pandas as pd
import snscrape.modules.twitter as sntwitter # the magic
import warnings
warnings.filterwarnings('ignore')

In [11]:
#setting up the directory
#root_dir = os.path.abspath(os.path.join(os.path.dirname("."), '.'))
#output: /Users/sbp

In [12]:
#Extractor Module
class ExtractTweets:
    
    def __init__(self, 
                 minTweetCountPerDay=10, 
                 minRetweetCount=0,
                 minLikeCount=0, 
                 minFollowersCount=0, 
                 VerifiedStatus=None, 
                 saveBufferDuration=3600):
        """
        Accepts basic input params each of integer datatype, except for VerifiedStatus which accepts boolean or None.
        """
        self.start_timer = datetime.strptime(time.strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")
        self.min_tweet_count_perDay = minTweetCountPerDay
        self.minRetweetCount = minRetweetCount
        self.minLikeCount = minLikeCount
        self.minFollowersCount = minFollowersCount
        self.VerifiedStatus = VerifiedStatus                           
        self.tweets_df = pd.DataFrame(columns=['date', 'tweet', 'lang', 'retweetCount', 'likeCount', 'replyCount', 
                                               'username', 'user_followersCount','user_friendsCount', 'verifiedStatus', 
                                               'tweet_url', 'hastags', 'chr_count', 'topic'])
        self.save_buffer_duration = saveBufferDuration
        return

    def save_copy(self):
        """
        Saves a temp copy for restoration and prevent API time limit exceed error.
        
        :return:
        pandas dataframe containing twitter record-data.
        """
        data = self.tweets_df.reset_index(drop=True)
        data['date'] = data['date'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d'))
        for filename in os.listdir("."):
            if filename.endswith('local.csv'):
                os.remove(filename)
        data.to_csv("./save_{}_local.csv".format(data.date.max()), index=False)
        return data
        
    def getTweets(self, start_date, end_date, keywords):
        """
        Extracts historical twitter data.
        
        :params:
        start_date - str in "YYYY-MM-DD" format
        end_date - str in "YYYY-MM-DD" format
        keywords - list of tuples, 
            e.g, [('recession'), ('football, 'worldcup', 'fifa'), ('war', 'ukraine')]
            e.g. ['recession']
        
        :return:
        pandas dataframe with features as:
         date: Tweet Timestamp
         tweet: tweet content
         lang: language classifer used by parent api
         retweetCount: tweet retweeted count
         likeCount: tweet like count
         replyCount: number of replies to original tweet
         username: user who tweeted
         user_followersCount: number of followers user has (tells you how popular the avg tweets are)
         user_friendsCount: number of friends user has
         verifiedStatus: If the user is Verified or not (i.e. pays 8 bucks every month!)
         tweet_url: Link of original tweet (click and see)
         hastags: If any hastags were used (hastags are important for search and info retrieval)
         chr_count: number of english characters in the original tweet
         topic: keywords you used for searching tweets (kind of labels)
        """
        
        if not(isinstance(keywords, list) or isinstance(keywords, tuple)):
            raise Exception("Incorrect Input Format! Please pass a list")
        
        for topic in keywords:
            # for saving local copies every buffer_hour
            st_time = datetime.strptime(time.strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")
            date = pd.to_datetime(start_date, format='%Y-%m-%d')
            e_date = pd.to_datetime(end_date, format='%Y-%m-%d') + pd.to_timedelta(1, unit='d')
            if isinstance(topic, tuple) or isinstance(topic, list):
                topic = " ".join(topic)
            search_query = topic
            print("search_query:", search_query)
        
            while date != e_date:
                nxt_date = date + pd.to_timedelta(1, unit='d')
                content = '{} since:{} until:{} lang:en'.format(search_query, date.strftime('%Y-%m-%d'), nxt_date.strftime('%Y-%m-%d'))
                print(content)
                
                # check for save buffer duration (set to 1 Hr by default)
                delta_buffer = (datetime.strptime(time.strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S") - st_time).seconds
                if delta_buffer >= self.save_buffer_duration:
                    self.save_copy()
                    # reset buffer
                    st_time = datetime.strptime(time.strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")

                lst_tweets = []
                for counter, tweet in enumerate(sntwitter.TwitterSearchScraper(content).get_items()):
                    if counter+1 > self.min_tweet_count_perDay: 
                        break
                    if tweet.likeCount >= self.minLikeCount \
                        or tweet.retweetCount >= self.minRetweetCount \
                        or tweet.user.followersCount >= self.minFollowersCount \
                        or (tweet.user.verified and isinstance(tweet.user.verified, bool) and tweet.user.verified == self.VerifiedStatus):
                        
                        # ----------------------------------------------------------------
                        # Potential custom preprocessing module here: 
                        # 1. Simple and short: https://www.kaggle.com/code/zenbird01/pranjalpathak-semantic-clustering-v1-0/notebook
                        # 2. Advanced: ./NLP_basics_preprocessing_vectorization_similarity.ipynb
                        # 3. Best: Check github - https://github.com/pranzell/NLP_Tools
                        # ----------------------------------------------------------------
                        
                        lst_tweets.append([
                            tweet.date, 
                            tweet.content, 
                            tweet.lang,
                            tweet.retweetCount,
                            tweet.likeCount,
                            tweet.replyCount,
                            tweet.user.username, 
                            tweet.user.followersCount, 
                            tweet.user.friendsCount, 
                            tweet.user.verified,
                            tweet.url,
                            tweet.hashtags,
                            len(str(tweet.content).strip()),
                            topic])
                
                self.tweets_df = self.tweets_df.append(pd.DataFrame(lst_tweets, columns=self.tweets_df.columns))
                date = nxt_date
        
        print("\n\nTOTAL TIME TAKEN {} minutes".format(((datetime.strptime(time.strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S") - self.start_timer).seconds)/60.0))
        return self.save_copy()
    
    
    def preprocess_shortText(self, text_col):
        # refer to Preprocessing ipynb file
        # https://github.com/pranzell/NLP_Tools
        pass

In [13]:
#Configuration
minTweetCountPerDay=50
minRetweetCount=100
minLikeCount=100
minFollowersCount=200
VerifiedStatus=None
saveBufferDuration=3600 # in seconds

start_date = "2022-10-01"
end_date = "2023-01-31"

# list of tuples, or a list of single str items check function definition `getTweets()`
keywords = [('supplements'), ("Performance enhancing drugs"), ("proteins"), ("diet")]
#keywords = [('non veganism'), ('antivegan','exvegan','yestomeat','meatlover','antiveganism')]


In [14]:
#Execution
#keywords = [('non veganism'), ('antivegan','exvegan','yestomeat','meatlover','antiveganism')]

et = ExtractTweets(minTweetCountPerDay, minRetweetCount, minLikeCount, minFollowersCount, VerifiedStatus, saveBufferDuration)
twitter_data = et.getTweets(start_date, end_date, keywords)

search_query: supplements
supplements since:2022-10-01 until:2022-10-02 lang:en
supplements since:2022-10-02 until:2022-10-03 lang:en
supplements since:2022-10-03 until:2022-10-04 lang:en
supplements since:2022-10-04 until:2022-10-05 lang:en
supplements since:2022-10-05 until:2022-10-06 lang:en
supplements since:2022-10-06 until:2022-10-07 lang:en
supplements since:2022-10-07 until:2022-10-08 lang:en
supplements since:2022-10-08 until:2022-10-09 lang:en
supplements since:2022-10-09 until:2022-10-10 lang:en
supplements since:2022-10-10 until:2022-10-11 lang:en
supplements since:2022-10-11 until:2022-10-12 lang:en
supplements since:2022-10-12 until:2022-10-13 lang:en
supplements since:2022-10-13 until:2022-10-14 lang:en
supplements since:2022-10-14 until:2022-10-15 lang:en
supplements since:2022-10-15 until:2022-10-16 lang:en
supplements since:2022-10-16 until:2022-10-17 lang:en
supplements since:2022-10-17 until:2022-10-18 lang:en
supplements since:2022-10-18 until:2022-10-19 lang:en
su

In [15]:
print(twitter_data.shape)
twitter_data.head(3)

(13232, 14)


,date,tweet,lang,retweetCount,likeCount,replyCount,username,user_followersCount,user_friendsCount,verifiedStatus,tweet_url,hastags,chr_count,topic
0,2022-10-01,tongkat ali + mushroom blend + green blend + g...,en,0,0,0,WhoIsNo17,593,377,False,https://twitter.com/WhoIsNo17/status/157636119...,None,105,supplements
1,2022-10-01,@meatguyf @Ren_Chandler4 I also bought the Fre...,en,0,1,1,Elpico72,253,4934,False,https://twitter.com/Elpico72/status/1576359757...,None,254,supplements
2,2022-10-01,@danluu No history of doping/supplements in re...,en,0,1,2,jaseemabid,3863,2473,False,https://twitter.com/jaseemabid/status/15763596...,None,124,supplements


In [16]:
twitter_data.to_csv('/Users/sbp/Downloads/twitterdatav1.csv', index=False)

In [ ]:
def read_copy(path=""):
    for f in os.listdir(path):
        if f.endswith('local.csv'):
            df = pd.read_csv(f, lineterminator='\n')
            df.hastags = df.hastags.apply(lambda x: literal_eval(x) if str(x) not in ['none', 'nan', 'np.nan', 'null', ''] else None)
            return df